In [1]:

pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pymysql
import pandas as pd

def connect_to_database(host, user, password, database):
    """Establish a connection to the MySQL database."""
    return pymysql.connect(host=host, user=user, password=password, database=database)

def create_table_if_not_exists(cursor):
    """Create the financial_statements table if it doesn't exist."""
    create_table_sql = """
    CREATE TABLE IF NOT EXISTS financial_statements (
        Year INT,
        Company VARCHAR(255),
        Category VARCHAR(100),
        Market_Cap FLOAT,
        Revenue FLOAT,
        Gross_Profit FLOAT,
        Net_Income FLOAT,
        Earnings_Per_Share FLOAT,
        EBITDA FLOAT,
        Shareholder_Equity FLOAT,
        Cash_Flow_Operating FLOAT,
        Cash_Flow_Investing FLOAT,
        Cash_Flow_Financial FLOAT,
        Current_Ratio FLOAT,
        Debt_Equity_Ratio FLOAT,
        ROE FLOAT,
        ROA FLOAT,
        ROI FLOAT,
        Net_Profit_Margin FLOAT,
        Free_Cash_Flow_Per_Share FLOAT,
        Return_Tangible_Equity FLOAT,
        Number_Employees INT,
        Inflation_Rate FLOAT
    );
    """
    cursor.execute(create_table_sql)
    print("Ensured the financial_statements table exists.")

def insert_data(cursor, sql, data):
    """Insert data into the database."""
    try:
        cursor.executemany(sql, data)
        print("Data inserted successfully.")
    except pymysql.MySQLError as e:
        print(f"Error inserting data: {e}")

def main():
    # Database connection parameters
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'password': 'password',
        'database': 'DW2'
    }

    try:
        # Connect to the database
        connection = connect_to_database(**db_config)
        print("Connection successful!")

        # Create a cursor object
        cursor = connection.cursor()

        # Ensure the table exists
        create_table_if_not_exists(cursor)

        # Load data from CSV file
        df_financials = pd.read_csv('/Users/shriyog/Documents/SEM5/DW/Assignment 4/C2P2 Project 1 Financial Statements.csv')  # Adjust the file path as needed

        # Transform: Clean and prepare the data
        df_financials.dropna(inplace=True)  # Remove missing values
        df_financials['Company'] = df_financials['Company'].str.strip()  # Trim whitespace

        # SQL query to insert data into financial_statements table
        financial_insert_sql = """
            INSERT INTO financial_statements (
                Year, Company, Category, Market_Cap, Revenue, Gross_Profit, 
                Net_Income, Earnings_Per_Share, EBITDA, Shareholder_Equity, 
                Cash_Flow_Operating, Cash_Flow_Investing, Cash_Flow_Financial, 
                Current_Ratio, Debt_Equity_Ratio, ROE, ROA, ROI, 
                Net_Profit_Margin, Free_Cash_Flow_Per_Share, Return_Tangible_Equity, 
                Number_Employees, Inflation_Rate
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        # Prepare data for insertion
        financial_data = [
            (
                row['Year'], row['Company'], row['Category'], row['Market Cap(in B USD)'],
                row['Revenue'], row['Gross Profit'], row['Net Income'],
                row['Earning Per Share'], row['EBITDA'], row['Share Holder Equity'],
                row['Cash Flow from Operating'], row['Cash Flow from Investing'],
                row['Cash Flow from Financial Activities'], row['Current Ratio'],
                row['Debt/Equity Ratio'], row['ROE'], row['ROA'],
                row['ROI'], row['Net Profit Margin'], row['Free Cash Flow per Share'],
                row['Return on Tangible Equity'], row['Number of Employees'],
                row['Inflation Rate(in US)']
            )
            for index, row in df_financials.iterrows()
        ]
        
        # Insert financial data
        insert_data(cursor, financial_insert_sql, financial_data)

        # Commit the transaction
        connection.commit()
        print("ETL process completed successfully!")

    except pymysql.MySQLError as e:
        print(f"Error: {e}")

    finally:
        # Ensure resources are cleaned up
        if 'cursor' in locals():
            cursor.close()
        if 'connection' in locals():
            connection.close()
        print("Database connection closed.")

if __name__ == "__main__":
    main()


Connection successful!
Ensured the financial_statements table exists.
Data inserted successfully.
ETL process completed successfully!
Database connection closed.
